## Clase 3: Hello (Machine Learning) World!

In [1]:
import pandas as pd

Importamos el dataset que habíamos trabajado previamente. Recordemos que este dataset tiene sólo anuncios de departamentos en venta de la ciudad de Buenos Aires. 

In [2]:
df = pd.read_csv("../Data/bsas_subset.csv")

In [3]:
print("¿Cómo se lee df.shape: ", df.shape, " ?")

¿Cómo se lee df.shape:  (7411, 15)  ?


In [4]:
print("Recordemos cuáles son las columnas que nos quedaron...\n")
df.info()

Recordemos cuáles son las columnas que nos quedaron...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7411 entries, 0 to 7410
Data columns (total 15 columns):
Unnamed: 0               7411 non-null int64
property_type            7411 non-null object
lat                      5474 non-null float64
lon                      5474 non-null float64
price                    7411 non-null float64
surface_total_in_m2      6202 non-null float64
surface_covered_in_m2    7058 non-null float64
price_usd_per_m2         6198 non-null float64
price_per_m2             7057 non-null float64
floor                    829 non-null float64
rooms                    3592 non-null float64
expenses                 2087 non-null float64
description              7411 non-null object
title                    7411 non-null object
ciudad                   7411 non-null object
dtypes: float64(10), int64(1), object(4)
memory usage: 868.6+ KB


Vamos a eliminar algunas columnas... "Unnamed: 0", "property_type" (porque sabemos que son todos departamentos), "price_usd_per_m2", "price_per_m2", "floor", "expenses", "title".

In [5]:
df = df.drop([ "Unnamed: 0", "property_type", "price_usd_per_m2", "price_per_m2", "floor", "expenses", "title"], axis=1)

Ahora bien, nos interesa poder calcular nosotros mismos el precio por metro cuadrado en dólares, pero vemos que hay más "surface_covered_in_m2" que "surface_total_in_m2". Por ello, vamos a imputar los casos nulos de "surface_total_in_m2" con "surface_covered_in_m2", cuando exista.

In [6]:
df.loc[df["surface_total_in_m2"].isnull(), "surface_total_in_m2"] = df.loc[df["surface_total_in_m2"].isnull(), "surface_covered_in_m2"]

Veamos cuántos casos nulos nos quedaron...

In [7]:
df["surface_total_in_m2"].isnull().value_counts()

False    7339
True       72
Name: surface_total_in_m2, dtype: int64

## Usando Regex

Las regex son patrones que siguen los strings y nos sirven para extraer información relevante. En este caso las vamos a usar para poder completar los datos faltantes de los ambientes. 

In [8]:
# re es el paquete que se usa para usar expresiones regulares
import re
def match_pattern(pattern, string):
    # Esta función permite ejecuta un patrón regex sobre un string y devuelve una lista
    result = re.findall(pattern, string)
    # Si la lista está vacía reemplazar [] por None
    if len(result) == 0:
        result = None
    # Si la lista tiene un integer adentro, por ejemplo, [3], guardar eso en result
    elif len(result) == 1:
        result = float(result[0])
    else:
    # Si la lista tiene muchos match (no debería pasar), reemplazar por None para evitar errores
        result = None
    return result

In [9]:
pattern = "([1-4]).*amb"

In [10]:
rooms_matched = df["description"].apply(lambda x: match_pattern(pattern, x))

In [11]:
df["rooms"][df["rooms"].isnull()] = rooms_matched[df["rooms"].isnull()]

/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df["rooms"].isnull().value_counts()

False    5445
True     1966
Name: rooms, dtype: int64

## Completando otros faltantes

Las regresiones lineales no pueden manejar casos faltantes, así que tendremos que imputar datos. Para ello usamos regex para encontrar la mayor cantidad de casos posibles pero vemos que hay 5800 que aún no fueron completados. Para eso vamos a usar una función de Pandas.

Vemos que algunas propiedades tienen demasiados cuartos...

In [13]:
df.rooms.value_counts()

2.0     1540
3.0     1355
1.0     1290
4.0      963
5.0      224
6.0       42
7.0       22
8.0        6
9.0        2
15.0       1
Name: rooms, dtype: int64

A los que tienen más de 5 cuartos les vamos a imputar 5, para que la regresión no se vea tan afectada por estos valores.

In [14]:
df.loc[df["rooms"]>5, "rooms"] = 5

Aunque en general es una mala prática vamos a imputar con la media.

In [15]:
df.rooms.mean()

2.529292929292929

In [16]:
df["rooms"] = df["rooms"].fillna(df.rooms.mean())

In [17]:
df.ciudad.value_counts()

Palermo      3187
Belgrano     2338
Caballito    1886
Name: ciudad, dtype: int64

## Eliminando observaciones

Vamos a eliminar aquellas filas que tienen observaciones nulas en alguna de las variables.

In [18]:
df = df.dropna()

In [19]:
df.shape

(5270, 8)

## Generando variable dummy

Vamos a transformar la variable ciudad en una variable que indica a qué barrio pertenece la observación. Es decir, vamos a crear una variable que va a valer 1 si la observación proviene de ese barrio o 0 si no.

Una de las tres las vamos a borrar para que no afecte a la regresión lineal.

In [20]:
dummies = pd.get_dummies(df.ciudad,drop_first=True)

In [21]:
dummies

,Caballito,Palermo
0,0,0
1,0,0
4,0,1
5,0,1
6,1,0
7,1,0
8,1,0
9,1,0
10,1,0
11,0,0


## Generando versión final del df

In [22]:
df.columns

Index(['lat', 'lon', 'price', 'surface_total_in_m2', 'surface_covered_in_m2',
       'rooms', 'description', 'ciudad'],
      dtype='object')

Vamos a borrar description y ciudad ya que no nos sirven más. Y price que es nuestro target lo vamos a guardar en y.

In [23]:
y = df.price

In [24]:
df = df.drop(["description", "ciudad", "price"], axis = 1)

Además, vamos a concatenar las variables dummies que creamos más arriba

In [25]:
df = pd.concat([df,dummies], axis = 1)

In [26]:
X = df

## Generando nuestro primer modelo

In [52]:
# sklearn es el paquete habitualmente usado en Machine Learning y linear_model es el módulo donde está el modelo
from sklearn.linear_model import LinearRegression

In [53]:
# Instanciamos el modelo
model = LinearRegression()

In [54]:
# Ajustamos el modelo y aprendemos los parámetros
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
# Vamos a predecir sobre los mismos datos para estimar la performance del modelo
y_predicted = model.predict(X)

In [39]:
# Calculamos el R2 y el error absoluto medio
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y, y_predicted))
print(mean_absolute_error(y, y_predicted))

0.31910195186060863
136241.2633651727


¿Qué opinan de estos resultados?

Ahora probemos lo siguiente, ¿si acotamos el alcance del modelo a propiedades entre cierto rango de metros?

In [60]:
print("Actualmente los metros máximos son:", max(X.surface_covered_in_m2), " y los mínimos son:" ,min(X.surface_covered_in_m2))

Actualmente los metros máximos son: 7029.0  y los mínimos son: 0.0


Vamos a eliminar las observaciones con menos de 25 metros cuadrados y más de 200.

In [69]:
y_2 = y[(X.surface_total_in_m2 > 25) & (X.surface_total_in_m2 < 200)]
X_2 = X[(X.surface_total_in_m2 > 25) & (X.surface_total_in_m2 < 200)]

In [68]:
# Volvemos a ajustar el modelo
model.fit(X_2, y_2)

In [70]:
# Volvemos a predecir
y_predicted = model.predict(X_2)

In [72]:
# obtenemos los resultados
print(r2_score(y_2, y_predicted))
print(mean_absolute_error(y_2, y_predicted))

0.7046104960173922
50621.91474648752


¿Qué opinan de los resultados ahora? ¿Qué cosas se les ocurre para mejorar el modelo?